<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BeautifulSoup" data-toc-modified-id="BeautifulSoup-1">BeautifulSoup</a></span><ul class="toc-item"><li><span><a href="#BeautifulSoup-설치" data-toc-modified-id="BeautifulSoup-설치-1.1">BeautifulSoup 설치</a></span></li><li><span><a href="#BeautifulSoup-import-하기" data-toc-modified-id="BeautifulSoup-import-하기-1.2">BeautifulSoup import 하기</a></span></li><li><span><a href="#BeautifulSoup-객체-생성-및-parser-사용" data-toc-modified-id="BeautifulSoup-객체-생성-및-parser-사용-1.3">BeautifulSoup 객체 생성 및 parser 사용</a></span></li><li><span><a href="#jupyter-notebook-Magic-Command-:-whos" data-toc-modified-id="jupyter-notebook-Magic-Command-:-whos-1.4">jupyter notebook Magic Command : whos</a></span></li><li><span><a href="#Selector-기본" data-toc-modified-id="Selector-기본-1.5">Selector 기본</a></span></li><li><span><a href="#Selector-실습" data-toc-modified-id="Selector-실습-1.6">Selector 실습</a></span><ul class="toc-item"><li><span><a href="#html.parser" data-toc-modified-id="html.parser-1.6.1">html.parser</a></span></li><li><span><a href="#lxml-parser" data-toc-modified-id="lxml-parser-1.6.2">lxml parser</a></span></li><li><span><a href="#부모자식-tag-선택하기(find_parent,-find_all),-recursive-동작-비교" data-toc-modified-id="부모자식-tag-선택하기(find_parent,-find_all),-recursive-동작-비교-1.6.3">부모자식 tag 선택하기(find_parent, find_all), recursive 동작 비교</a></span></li><li><span><a href="#좌우-tag-선택하기-(find_next_sibling,-find_previous_-sibling)" data-toc-modified-id="좌우-tag-선택하기-(find_next_sibling,-find_previous_-sibling)-1.6.4">좌우 tag 선택하기 (find_next_sibling, find_previous_ sibling)</a></span></li><li><span><a href="#선물-가격정보-추출하기" data-toc-modified-id="선물-가격정보-추출하기-1.6.5">선물 가격정보 추출하기</a></span></li></ul></li><li><span><a href="#구글-검색결과-추출하기" data-toc-modified-id="구글-검색결과-추출하기-1.7">구글 검색결과 추출하기</a></span></li><li><span><a href="#네이버-검색결과-추출하기-(숙제1)" data-toc-modified-id="네이버-검색결과-추출하기-(숙제1)-1.8">네이버 검색결과 추출하기 (숙제1)</a></span></li><li><span><a href="#다음-검색결과-추출하기-(숙제2)" data-toc-modified-id="다음-검색결과-추출하기-(숙제2)-1.9">다음 검색결과 추출하기 (숙제2)</a></span></li><li><span><a href="#[기타]-DOM-Tree-탐색을-지원하는-함수-만들어-보기-(To-Do)" data-toc-modified-id="[기타]-DOM-Tree-탐색을-지원하는-함수-만들어-보기-(To-Do)-1.10">[기타] DOM Tree 탐색을 지원하는 함수 만들어 보기 (To-Do)</a></span></li></ul></li></ul></div>

"05.웹 크롤링 - DOM.pdf"(day5) 강의 내용에 대한 실습 코드 입니다.

## BeautifulSoup

이전 강의에서 정의한 getDownload 함수를 사용해서 웹페이지를 다운로드 해봅니다.

In [1]:
import requests

header = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36"}
base_retries = 3

def getDownload(url, param=None, retries=base_retries):
    resp = None

    try:
        resp = requests.get(url, params=param, headers=header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print("Retries: {0}".format(base_retries - retries + 1))
            return getDownload(url, param, retries - 1)
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.headers)
    
    return resp

In [2]:
url = "https://www.google.com/search"
param = {"q":"파이썬"}

html = getDownload(url, param=param)
# html.text

In [3]:
type(html)    # Response 객체

requests.models.Response

In [4]:
type(html.text)  # string

str

다운로드 한 html.text가 위에서 보는 것처럼 string이기 때문에 직접 조작하기 어렵습니다.

DOM tree를 구성해서 조작하기 쉽도록 해주는 **`BeautifulSoup`** 패키지를 사용 합니다.

### BeautifulSoup 설치

In [5]:
# !conda install beautifulsoup4
# !pip install beautifulsoup4

### BeautifulSoup import 하기

설치한 패키지명은 `beautifulsoup4` 이지만 import 할 패키지명은 `bs4` 입니다.

In [6]:
from bs4 import BeautifulSoup

### BeautifulSoup 객체 생성 및 parser 사용

HTML은 XML과 달리 **`Not Well-formed`** 이기 때문에 자동으로 분석하기 어렵다.

HTML 웹 페이지의 흔한 오류들...
- 닫는 태그가 없는 경우가 있다.
- elements가 overlap 된 경우가 있다.
- 여는 태그와 닫는 태그의 case(대소문자)가 일치하지 않는 경우가 있다.

속도와 오류처리 성능 등을 고려한 `parser` 선택이 중요하다. ( **`lxml parser`** 를 추천함 )
- python의 html.parser
- lxml의 html.parser
- lxml의 xml.parser
- html5lib

parser 들을 사용 해보자.

In [7]:
# 테스트를 위한 간단한 html 페이지를 만든다. (Well-formed)
html = """
<document>
<html>
    <head></head>
    <body>
        <div>
            <p>
                <a>Go to page</a>
            </p>
        </div>
    </body>
</document>
"""

In [8]:
dom = BeautifulSoup(html, "lxml")    # features 파라미터에 parser를 지정 (lxml parser 사용)

In [9]:
type(dom)

bs4.BeautifulSoup

In [10]:
dom.html.head   # 태그로 접근 가능함

In [11]:
type(dom.html)

bs4.element.Tag

In [12]:
type(dom.html.body)

bs4.element.Tag

BeautifulSoup은 DOM Tree를 반환하며, 이 객체는 tag 명칭과 값의 쌍으로 이루어진 **`Tag 객체`** 들로 이루어져 있다.

In [13]:
dom.html

<html><body><document>
<div>
<p>
<a>Go to page</a>
</p>
</div>
</document>
</body></html>

In [14]:
dom.prettify()   # 들여쓰기를 해서 보기좋게 출력함

'<html>\n <body>\n  <document>\n   <div>\n    <p>\n     <a>\n      Go to page\n     </a>\n    </p>\n   </div>\n  </document>\n </body>\n</html>'

In [15]:
dom.html.body.div.p.a

<a>Go to page</a>

In [16]:
dom.html.body.div.p.a.text

'Go to page'

In [17]:
dom.a.text

'Go to page'

추가적으로 `name`, `attr`, `text` 등 속성도 사용해볼것 (dom.a.attr, ...)

BeautifulSoup의 DOM Tree는 위에서 처럼, <br>
Tag를 chain 형태로 연결해서 단계적으로 접근하거나, 특정 Tag를 바로 접근도 가능하다.

### jupyter notebook Magic Command : whos

In [18]:
%whos

Variable        Type             Data/Info
------------------------------------------
BeautifulSoup   type             <class 'bs4.BeautifulSoup'>
autopep8        module           <module 'autopep8' from '<...>te-packages/autopep8.py'>
base_retries    int              3
dom             BeautifulSoup    <html><body><document>\n<<...>document>\n</body></html>
getDownload     function         <function getDownload at 0x10eb04a60>
header          dict             n=1
html            str              \n<document>\n<html>\n   <...>   </body>\n</document>\n
json            module           <module 'json' from '/Use<...>hon3.6/json/__init__.py'>
param           dict             n=1
requests        module           <module 'requests' from '<...>es/requests/__init__.py'>
url             str              https://www.google.com/search


### Selector 기본

In [19]:
# <a> 태그를 하나 더 추가해본다.
html = """
<document>
<html>
    <head></head>
    <body>
        <div>
            <p>
                <a>Go to page1</a>
                <a>Go to page2</a>
            </p>
        </div>
    </body>
</document>
"""

In [20]:
dom = BeautifulSoup(html, "lxml")

In [21]:
dom.a

<a>Go to page1</a>

In [22]:
dom.find('a')

<a>Go to page1</a>

In [23]:
dom.find_all('a')   # BeautifulSoup이 버전업 되면서, findAll() 형태의 메서드는 없어지고, find_all() 형태의 메서드로 대체되었음

[<a>Go to page1</a>, <a>Go to page2</a>]

위의 방법으로는 두번째 `<a>` 태그 만 가져오지는 못한다. 

태그에 `id`와 `class`를 추가해본다.

In [24]:
html = """
<document>
<html>
    <head></head>
    <body>
        <div id="result">
            <p class="row">
                <a class="red">Go to page1</a>
                <a class="blue">Go to page2</a>
            </p>
        </div>
    </body>
</document>
"""

In [25]:
dom = BeautifulSoup(html, "lxml")

In [26]:
import bs4   # bs4에서는 기본적으로 Tag의 속성들을 제어할 수 있는 다양한 메서드를 제공함

# bs4.element.Tag.get()
# bs4.element.Tag == DOM.Elements

In [27]:
dom.a, dom.find('a')

(<a class="red">Go to page1</a>, <a class="red">Go to page1</a>)

In [28]:
for tag in dom.find_all('a'):
    print(tag.name, tag["class"], tag.text)

a ['red'] Go to page1
a ['blue'] Go to page2


태그에 ID와 Class를 지정해줬기 때문에 아래처럼 특정 element에 직접 접근이 가능하다.

In [29]:
dom.find_all('a', {"class":"blue"})

[<a class="blue">Go to page2</a>]

In [30]:
dom.find_all(["div", "a"])   # tag는 여러개를 한꺼번에 지정할 수 있다.(튜플, 리스트, Set 등)

[<div id="result">
 <p class="row">
 <a class="red">Go to page1</a>
 <a class="blue">Go to page2</a>
 </p>
 </div>, <a class="red">Go to page1</a>, <a class="blue">Go to page2</a>]

In [31]:
dom.find_all("", {"id":"result"})     # tag를 지정하지 않고, 속성만으로 찾을 수도 있다.

[<div id="result">
 <p class="row">
 <a class="red">Go to page1</a>
 <a class="blue">Go to page2</a>
 </p>
 </div>]

In [32]:
dom.find_all("a", recursive=False)
# dom.find_all("a", recursive=True)    # default는 True

[]

find_all() 함수에 recursive=False로 지정하면 특정 태그를 direct로 찾지 못한다. direct children 만 찾을 수 있다.

In [33]:
dom.find_all("html", recursive=False)

[<html><body><document>
 <div id="result">
 <p class="row">
 <a class="red">Go to page1</a>
 <a class="blue">Go to page2</a>
 </p>
 </div>
 </document>
 </body></html>]

### Selector 실습

http://pythonscraping.com/pages/page3.html 사이트에서 Selector 실습하기

![DOM Exercises](./images/selector_exercises.png)

In [34]:
url = "http://pythonscraping.com/pages/page3.html"
html = getDownload(url, param=param)

#### html.parser

In [35]:
dom = BeautifulSoup(html.content, "html.parser")

In [36]:
footer = dom.find("div", {"id":"footer"})
footer

<div id="footer">
© Totally Normal Gifts, Inc. <br/>
+234 (617) 863-0736
</div>

In [37]:
parent = footer.find_parent()
parent.name, parent.attrs

('[document]', {})

#### lxml parser

In [38]:
dom = BeautifulSoup(html.content, "lxml")

In [39]:
footer = dom.find("div", {"id":"footer"})
footer

<div id="footer">
© Totally Normal Gifts, Inc. <br/>
+234 (617) 863-0736
</div>

In [40]:
parent = footer.find_parent()
parent.name, parent.attrs

('div', {'id': 'wrapper'})

In [41]:
parents = footer.find_parents()
for row in parents:
    print(row.name, row.attrs)

div {'id': 'wrapper'}
body {}
html {}
[document] {}


lxml parser는 find_parent() 결과가 정상적이지만, html.parser는 tag를 찾지 못했습니다.<br>
parser 마다 특성이 있습니다. 여기서는 lxml parser를 사용합니다.

#### 부모자식 tag 선택하기(find_parent, find_all), recursive 동작 비교

In [42]:
children = parent.find_all()    # default는 recursive=False

# for row in children:
#     print(row.name, row.attrs)

위에서 처럼  자식 노드에서 부모 노드로 이동 후 전체 태그를 찾는 `find_all()` 메서드 결과를 보면, <br>
원래의 children tag의 하부 tree 뿐 아니라 형제 Tree의 하부 까지 모두 찾는다.

`recursive=False`로 지정하면, 원래의 children tag의 하부 Tree 만 찾는다.

In [43]:
children = parent.find_all(recursive=False)
for row in children:
    print(row.name, row.attrs)

img {'src': '../img/gifts/logo.jpg', 'style': 'float:left;'}
h1 {}
div {'id': 'content'}
table {'id': 'giftList'}
div {'id': 'footer'}


#### 좌우 tag 선택하기 (find_next_sibling, find_previous_ sibling)

In [44]:
descendants = parent.find_all()

# for row in descendants:
#     print(row.name, row.attrs)

In [45]:
divChildren = children[2].find_all(recursive=False)

for row in divChildren:
    print(row.name, row.attrs, row.text)

p {} 
We haven't figured out how to make online shopping carts yet, but you can send us a check to:
123 Main St.
Abuja, Nigeria
We will then send your totally amazing gift, pronto! Please include an extra $5.00 for gift wrapping.


In [46]:
divTag = children[2]

# children[1] == divTag.find_previous_sibling()
children[1].name, divTag.find_previous_sibling().name

('h1', 'h1')

In [47]:
children[3] == divTag.find_next_sibling()

True

#### 선물 가격정보 추출하기

찾고자 하는 정보를 바로 찾을 수 있으면 좋지만, <br>
그렇지 않으면 상위의 태그 중에 id나 class가 명확히 구분되는 태그를 찾아서 tree를 탐색한다.

선물 리스트가 테이블에 담겨있고, 각각의 선물 리스트는 "gift"라는 class명을 가지고 있다.<br>
각각의 선물 리스트는 4개의 td 태그로 되어 있고, 3번째에 cost 정보가 포함되어 있다.

In [48]:
gifts = dom.find_all("", {"class":"gift"})

costs = []

for gift in gifts:
    tds = gift.find_all(recursive=False)    # 각각의 gift별 td 태그 리스트
    costs.append(tds[2].text.strip())

print(costs)

['$15.00', '$10,000.52', '$10,005.00', '$0.50', '$1.50']


### 구글 검색결과 추출하기

구글 검색을 통해 "파이썬 자습서" 라는 문자열의 검색결과 중 제목 부분을 추출해봅니다.

`우선 브라우저의 "페이지 소스 보기" 또는 "검사" 기능을 이용해서, 검색결과에서 찾고자 하는 문자열을 탐색 해봅니다.`

제목 자체의 class명은 검색 리스트 마다 다르지만,<br>
`제목 위쪽에 있는 div 태그의 class명이 "r"로 일정한 것을 발견할 수 있습니다.`  (찾기가 쉽지는 않습니다.)

In [49]:
url = "http://www.google.com/search"
param = {"q":"파이썬 자습서"}

html = getDownload(url, param)
dom = BeautifulSoup(html.text, "lxml")

# for tag in dom.find_all("", {"class":"r"}):
#     print(tag.prettify())

`<div class="r"> 태그 하부에 <a> 태그의 text 부분에 제목이 보입니다.`(a 태그의 text 중 실제 제목은 h3 태그에 있습니다.)

In [50]:
for tag in dom.find_all("div", {"class":"r"}):
    print(tag.find("a").h3.text)

파이썬 자습서 — Python 3.7.2 documentation
프로그래머가 아닌 이들을 위한 파이썬 3 자습서 - 위키책
byte of python(한국어판 PDF) - SourceForge
파이썬 강좌 및 유용한 사이트 모음 : 네이버 블로그
Visual Studio 자습서 1단계의 Python 프로젝트 만들기 | Microsoft Docs
Python | 자습서, API, SDK, 설명서 | AWS 개발자 센터 - Amazon.com
5.2 파이썬(Python) 시작하기 – Computational Thinking
무료 파이썬 자습서, eBook 및 전자책 - Blue Breeze


브라우저의 검색된 결과와 비교해보면, 검색 링크가 있는 모든 제목이 출력되었습니다.

`구글 검색결과에서 제목을 추출하는 전체 코드는 아래와 같습니다.`
```
url = "http://www.google.com/search"

html = getDownload(url, {"q":"파이썬 자습서"})
dom = BeautifulSoup(html.text, "lxml")
tags = dom.find_all("div", {"class":"r"})

for tag in tags:
    print(tag.find("a").h3.text)
```

### 네이버 검색결과 추출하기 (숙제1)

[네이버](http://www.naver.com)에서 "파이썬 자습서"을 검색 후 검색결과 브라우저의 URL은 아래와 같습니다.

`https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=파이썬+길잡이&oquery=파이썬+길잡이&tqi=U45jMwpVuECsssHReO0ssssssfd-106804`

URL을 주소부분과 Query String(parameter)으로 분리 합니다. Query String 중 query 값만 필수로 전달되면 됩니다.

In [52]:
url = "https://search.naver.com/search.naver"

# param = {
#     "sm": "tab_hty.top",
#     "where": "nexearch",
#     "query": "파이썬 자습서",
#     "oquery": "파이썬 자습서",
#     "tqi": "U45jMwpVuECsssHReO0ssssssfd-106804"
# }
param = {
    "query": "파이썬 자습서",
}

html = getDownload(url, param)
# html.text

검색결과 브라우저에서 "검사" 기능을 이용해 Tree를 탐색해봅니다.

`전체 검색결과는 class이 "main_pack"인 div 태그 하부에 존재 합니다.`<br>
참고로, 검색결과 우측의 뉴스토픽은 class명이 "sub_pack"인 div 태그 내에 존재 합니다.

검색결과는 블로그, 카페, 동영상, 뉴스, 지식인, 웹사이트, 이미지 섹션으로 구분되어 있고,<br>
이중에 동영상, 이미지 섹션을 제외하고는 공통적으로 `ul 태그의 class명이 "type01"` 이고,<br>
`ul 태그 아래에 li 태그 내에 세부 리스트가 있습니다.`<br>
참고로, 블로그 만 검색하려면 class명 `sh_blog_top`를, 카페 만 검색하려면 class명 `sh_cafe_top`를 사용할 수 있습니다.

검색결과 리스트의 제목은 `ul -> li -> dl -> dt -> a` 태그 구조로 되어 있고, a 태그의 text가 제목 문자열 입니다.

In [53]:
# 전체 검색결과를 확인해봅니다.
dom = BeautifulSoup(html.text, "lxml")

divTag = dom.find_all("", {"class":"main_pack"})

# for tag in divTag:
#     print(tag.prettify())

위의 결과는 불필요한 정보가 너무 많습니다.

동영상, 이미지를 제외하고, class명이 type01인 섹션의 검색결과들을 탐색해봅니다.

In [54]:
ulTags = dom.find_all("", {"class":"type01"})

# for ul in ulTags:
#     print(ul.prettify())
#     print("-" * 50)

In [55]:
for ul in ulTags:
    
    # 참고로, a 태그를 직접 찾으면 제목 있는 위치의 a 태그 뿐 아니라, 기타 링크까지 모두 찾기 때문에 불필요한 정보가 추출됨
    # for a in ul.find_all("a"):
    #    print(a.text.strip())
    
    for dt in ul.find_all("dt"):    # dt 태그 아래에 있는 a 태크 만 찾아서, a 태그의 text 값을 출력합니다.
        print(dt.a.text.strip())
    
    print("-" * 50)    # 각 섹션별 구분자를 출력 합니다.

파이썬 자습서
[python][ebook] 귀도의 파이썬 자습서(2.5) (한글 EBook)
자료 - 파이썬 자습서
--------------------------------------------------
[파이선] 파이썬 자습서 (한글화가 잘 되어 있네요)
파이썬 ve C언어
과학 전공도서/자습서/교양서적 등 팝니다~
[nRF Mesh] Sending mesh packets [2-serial examples]
C언어를 배우기 전에 파이썬공부를 왜 하는가
--------------------------------------------------
파이썬, 주류 언어로 부상··· 티오베와 PyPL 지표에서 상위권
떠오르는 심층 신경망 API, "케라스" 알아보기
IT전문가를 위한 라즈베리 파이 프로젝트 13선
개발자 출신 빅토리아시크릿 모델 ... '뇌섹녀' 인증
기술력 향상을 도울 온라인 무료 교육 사이트 8선
--------------------------------------------------
올해 대학 합격하고 컴퓨터 공학과에
경북대학교 소프트웨어학과 진학에 관해서
고1 공부법 및 진로
--------------------------------------------------
The Python Tutorial — Python 3.7.2 documentation
프로그래머가 아닌 이들을 위한 파이썬 3 자습서
프로그래머가 아닌 이들을 위한 파이썬 3 자습서/재귀
--------------------------------------------------


원하는 결과가 정상적으로 출력되었습니다.

`네이버의 검색결과에서 제목을 추출하는 전체 코드는 아래와 같습니다.`
```
url = "https://search.naver.com/search.naver"

html = getDownload(url, {"query": "파이썬 자습서"})
dom = BeautifulSoup(html.text, "lxml")
ulTags = dom.find_all("", {"class":"type01"})

for ul in ulTags:
    for dt in ul.find_all("dt"):
        print(dt.a.text)
    print("-" * 50)
```

### 다음 검색결과 추출하기 (숙제2)

[다음](https://www.daum.net)에서 "파이썬 자습서"을 검색 후 검색결과 브라우저의 URL은 아래와 같습니다.

`https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&q=파이썬+자습서`

다른 파라미터를 제거하고 q 값만 아래와 같이 전달해서 검색해보면, 정상적으로 검색 됩니다.

`https://search.daum.net/search?q=파이썬+자습서`

다른 파라미터를 제거해보니, 프리미엄링크(광고) 카테고리가 없어져서 더 좋아보입니다.

우선 위의 URL을 주소부분과 Query String("q" parameter) 부분으로 분리 합니다.

In [56]:
# 다음 포탈에서는 user-agent를 약식으로 기술하면, 로봇으로 의심해서 캡차 사이트로 redirect 됨
# header = {"user-agent": "Mozilla/5.0"}      # 약식으로 기술하면 정상적으로 검색되지 않음
header = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36"}

url = "https://search.daum.net/search"
# param = {
#     "w": "tot", 
#     "DA": "YZR", 
#     "t__nil_searchbox": "btn", 
#     "sug": "", 
#     "sugo": "", 
#     "q": "파이썬 자습서"
# }
param = {"q": "파이썬 자습서"}

html = getDownload(url, param)
# html.text

검색결과 브라우저에서 "검사" 기능을 이용해 Tree를 탐색해봅니다.

`전체 검색결과는 class이 "inner_article"인 div 태그 하부에 존재 합니다.`<br>
참고로, 검색결과 우측의 실시간 이슈/뉴스 검색어는 class명이 "g_comp_s"인 div 태그 내에 존재 합니다.

검색결과는 블로그, 웹문서, 브런치, 이미지, 뉴스, 카페글 섹션으로 구분되어 있고,<br>
다행이 `각 섹션의 div 태그의 class명이 "g_comp"로 동일 합니다.` 여기서 부터 시작하면 좋을 것 같습니다.

`세부 검색 리스트는 "g_comp" class 하부에 포함된 ul 태그 아래에 li 태그들 아래에 존재합니다.`

찾고자 하는 제목은 li 태그 아래에 a 태그에 있지만, a 태그까지 찾아가는 경로는 섹션마다 조금씩 다르고,<br>
li 태그 내에는 불필요한 a 태그도 존재합니다.

다행히도, 블로그, 웹문서, 뉴스, 카페글 섹션은 <br>
`제목을 나타내는 a 태그의 바로 위 div 태그의 class명이 "mg_tit"`로 동일 합니다.<br>
여기서는 `"mg_tit"` class명을 이용해서 블로그, 웹문서, 뉴스, 카페글에서만 제목을 추출해보겠습니다.<br>
참고로, 브런치의 제목도 추출 하려면 `"tit_cont"` class명으로 찾으면 됩니다.

In [57]:
dom = BeautifulSoup(html.text, "lxml")

# divTags = dom.find_all("", {"class": "g_comp"})
# for tag in divTags:
#     print(tag.prettify())

divTags = dom.find_all("", {"class": "mg_tit"})
for tag in divTags:
    print(tag.find("a").text.strip())

[파이썬기초] 자습서
무료 파이썬 자습서, eBook 및 전자책
[python][ebook] 귀도의 파이썬 자습서(2.5) (한글 EBook)
python자습서_ 숫자, 문자열
파이썬 자습서 ¶
【免費教育APP】파이썬 자습서|線上玩APP不花錢
프로그래머가 아닌 이들을 위한 파이썬 3 자습서/서문
프로그래머가 아닌 이들을 위한 파이썬 3 자습서/FAQ
떠오르는 심층 신경망 API, "케라스" 알아보기
파이썬, 주류 언어로 부상··· 티오베와 PyPL 지표에서 상위권
IT전문가를 위한 라즈베리 파이 프로젝트 13선
개발자 출신 빅토리아시크릿 모델 .. '뇌섹녀' 인증
파이썬 정리 5일차
어떻게 무료로 데이터 과학자가 될 수 있는지((주...
떠오르는 심층 신경망 API, "케라스" 알아보기
Genymtn


원하는 결과가 정상적으로 출력되었습니다.

`다음 포탈의 검색결과에서 제목을 추출하는 전체 코드는 아래와 같습니다.`
```
header = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.109 Safari/537.36"}

url = "https://search.daum.net/search"
param = {"q": "파이썬 자습서"}

html = getDownload(url, param)
dom = BeautifulSoup(html.text, "lxml")

divTags = dom.find_all("", {"class": "mg_tit"})
for tag in divTags:
    print(tag.find("a").text.strip())
```
user-agent 값은 자신의 컴퓨터 환경에 따라 다를 수 있습니다.

### [기타] DOM Tree 탐색을 지원하는 함수 만들어 보기 (To-Do)

학습 차원에서 DOM Tree 전체를 차례로 탐색하는 함수를 만들어 봅시다.<br>
tag 명칭과 클래스/ID를 추출해서 dom tree 구조를 출력해주는 함수를 만들어 사용하면 좋을 듯 합니다.

전체 tree 구조 중 원하는 tag 또는 class/id명을 기준으로 <br>
조건에 맞는 것 만 tree로 보여주는 기능도 유용할 듯 합니다.

DOM Tree 탐색 과정을 지원하는 기능이 도구가 있는지 찾아보고, <br>
탐색 과정을 자동화 하는 방안도 검토해봅시다.